## 情感分布考察

In [1]:
import os
import pandas as pd
import numpy as np
import paddlehub as hub

ImportError: cannot import name 'amp_state' from 'paddle.fluid.dygraph.amp.auto_cast' (C:\ProgramData\Anaconda3\lib\site-packages\paddle\fluid\dygraph\amp\auto_cast.py)

In [ ]:
df = pd.read_csv("xue_distance_final.csv")

In [ ]:
df = df['content']

In [ ]:
!pip3 install paddlehub==2.0.0